# Ensambles de árboles de decisión

## Gradient Boosting  

- Generaliza el concepto de boosting a cualquier función de costo derivable
- Cada clasificador en la cadena se entrena con los residuos del clasificador anterior

En scikit-learn en el módulo [`ensemble`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) se encuentra Gradient Boosting para clasificar y hacer regresión

    sklearn.ensemble.GradientBoostingClassifier(loss=’deviance’, learning_rate=0.1, 
                                                n_estimators=100, subsample=1.0, 
                                                max_depth=3, ...)
                                                
Esta implementación usa árboles como clasificador débil

Explicación de los parámetros (algunos):
- n_estimators: Número de árboles
- max_depth: Profundidad de los árboles
- subsample: Se usa para que cada árbol use una submuestra del dataset
- learning_rate: Se usa para disminuir la contribución de cada árbol sucesivo
- max_features: Número de atributos a considerar en cada split (reduce la varianza)


Encontremos el mejor ensamble usando 5-fold cross validation

In [ ]:
from sklearn import ensemble

params = {'loss':('deviance', 'exponential'), 
          'max_depth':[1, 5, 10, 20],
          'n_estimators': [1, 10, 20, 50, 100]}
model = ensemble.GradientBoostingClassifier(subsample=0.5, learning_rate=0.1, max_features=None)
gbs = GridSearchCV(model, params, cv=5)
gbs.fit(X_train, Y_train)

In [ ]:
means = gbs.cv_results_['mean_test_score'][gbs.cv_results_['param_loss']== 'deviance']
stds = gbs.cv_results_['std_test_score'][gbs.cv_results_['param_loss']== 'deviance']
for mean, std, params in zip(means, stds, gbs.cv_results_['params']):
    print("score: %0.3f (+/-%0.03f) con %r"
          % (mean, std * 2, params))

El mejor modelo es:

In [ ]:
model = gbs.best_estimator_
print(model)

Gradient Boosting funciona bien con árboles poco profundo

Clasificador débil con alto sesgo y baja varianza

In [ ]:
Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
fig, ax = plt.subplots(1, 2, figsize=(8, 3), tight_layout=True)

ax[0].contourf(xx, yy, Z.reshape(xx.shape), cmap=plt.cm.RdBu, alpha=0.5)
ax[0].scatter(X[Y==0, 0], X[Y==0, 1], color='k', s=10, marker='o', alpha=0.5)
ax[0].scatter(X[Y==1, 0], X[Y==1, 1], color='k', s=10, marker='x', alpha=0.5)
fpr, tpr, th = roc_curve(Y_train, model.predict_proba(X_train)[:, 1])
ax[1].plot(fpr, tpr, label='Entrenamiento', linewidth=2)
fpr, tpr, th = roc_curve(Y_test, model.predict_proba(X_test)[:, 1])
ax[1].plot(fpr, tpr, label='Prueba', linewidth=2)
plt.legend(loc=4)
plt.xlabel('FPR')
plt.ylabel('TPR')
ax[1].set_ylim([0.0, 1.0]);

Podemos comparar el rendimiento del mejor árbol con el mejor ensamble en el conjunto de Prueba

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 4), tight_layout=True)
ax.set_xlabel('FPR')
ax.set_ylabel('TPR/Recall')

Y_pred = dts.best_estimator_.predict_proba(X_test)[:, 1]
fpr, tpr, th = roc_curve(Y_test, Y_pred)
ax.plot(fpr, tpr, label="Decision Tree %0.4f" %auc(fpr, tpr), linewidth=2)

Y_pred = gbs.best_estimator_.predict_proba(X_test)[:, 1]
fpr, tpr, th = roc_curve(Y_test, Y_pred)
ax.plot(fpr, tpr, label="Gradient boosting %0.4f" %auc(fpr, tpr), linewidth=2)
plt.legend(loc=4);

## Random Forest

- Conjunto de árboles de decisión entrenados en paralelo usando bootstrap 
- Cada árbol se entrena con un **subconjunto aleatorio** de los datos (bagging)
- Cada árbol se entrena con un **subconjunto aleatorio** de los atributos (random forest)

Nuevamente en el módulo [`ensemble`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) podemos encontrar random forest para clasificar y hacer regresión

    sklearn.ensemble.RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, 
                                            max_depth=None,
                                            max_features=’auto’, bootstrap=True, 
                                            oob_score=False,
                                            n_jobs=None, class_weight=None, ...)
                                                

Explicación de los parámetros (algunos):
- n_estimators: Número de árboles
- max_depth: Profundidad de los árboles
- max_features: Número de atributos a considerar en cada split
- criterion: Para decidir como se escogen los splits
- bootstrap: Muestreo con reemplazo (desactiva bagging)
- class_weight: Usar o no una mayor ponderación para las clases menos representadas
- n_jobs: número de cores para entrenar




In [ ]:
params = {'criterion':('entropy', 'gini'),
          'max_depth':[1, 5, 10, 20],
          'n_estimators': [1, 10, 20, 50, 100]}
model = ensemble.RandomForestClassifier(max_features=None, n_jobs=-1)
rfs = GridSearchCV(model, params, cv=5)
rfs.fit(X_train, Y_train)
display(rfs.best_estimator_)
display(rfs.best_score_)

El mejor Random Forest es:

In [ ]:
model = rfs.best_estimator_
print(model)

A diferencia de GB, Random Forest prefiere árboles más profundos

Clasificadores débiles con bajo sesgo y alta varianza

In [ ]:
Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
fig, ax = plt.subplots(1, 2, figsize=(8, 3), tight_layout=True)

ax[0].contourf(xx, yy, Z.reshape(xx.shape), cmap=plt.cm.RdBu, alpha=0.5)
ax[0].scatter(X[Y==0, 0], X[Y==0, 1], color='k', s=10, marker='o', alpha=0.5)
ax[0].scatter(X[Y==1, 0], X[Y==1, 1], color='k', s=10, marker='x', alpha=0.5)
fpr, tpr, th = roc_curve(Y_train, model.predict_proba(X_train)[:, 1])
ax[1].plot(fpr, tpr, label='Entrenamiento', linewidth=2)
fpr, tpr, th = roc_curve(Y_test, model.predict_proba(X_test)[:, 1])
ax[1].plot(fpr, tpr, label='Prueba', linewidth=2)
plt.legend(loc=4)
plt.xlabel('FPR')
plt.ylabel('TPR')
ax[1].set_ylim([0.0, 1.0]);

Podemos comparar el rendimiento del mejor árbol con los mejores ensambles Prueba

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 4), tight_layout=True)
ax.set_xlabel('FPR')
ax.set_ylabel('TPR/Recall')

Y_pred = dts.best_estimator_.predict_proba(X_test)[:, 1]
fpr, tpr, th = roc_curve(Y_test, Y_pred)
ax.plot(fpr, tpr, label="Decision Tree %0.4f" %auc(fpr, tpr), linewidth=2)

Y_pred = gbs.best_estimator_.predict_proba(X_test)[:, 1]
fpr, tpr, th = roc_curve(Y_test, Y_pred)
ax.plot(fpr, tpr, label="Gradient boosting %0.4f" %auc(fpr, tpr), linewidth=2)

Y_pred = rfs.best_estimator_.predict_proba(X_test)[:, 1]
fpr, tpr, th = roc_curve(Y_test, Y_pred)
ax.plot(fpr, tpr, label="Random Forest %0.4f" %auc(fpr, tpr), linewidth=2)
plt.legend(loc=4);

## Más sobre ensambles

Dos algoritmos de Gradient Boosting para árboles de decisión (GBDT) extremadamente competitivos:

- [XGBoost](http://dmlc.cs.washington.edu/xgboost.html)
- [LightGBM](https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree) e [implementación oficial](https://lightgbm.readthedocs.io/en/latest/index.html)

Ambos implementan estrategias para mejorar la eficiencia y realizar cálculos paralelos/distribuidos e incluso usando GPU

Estado del arte en clasificación de datos estructurados (tablas)
